In [1]:
from pprint import pprint
from transformers import AutoTokenizer, pipeline
import transformers
import torch

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [2]:
input_text = ['Hi my name is Jalaj','I am learning Hugging Face.','I love Hugging Face']

In [3]:
# model_checkpoint = 'distilbert-base-uncased'
model_checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [4]:
tokenizer.vocab_size

30522

In [5]:
tokenizer.get_added_vocab()

{'[PAD]': 0, '[UNK]': 100, '[CLS]': 101, '[SEP]': 102, '[MASK]': 103}

In [6]:
!ls -lah /

total 10
drwxr-xr-x  22 root  wheel   704B Dec  7 13:41 .
drwxr-xr-x  22 root  wheel   704B Dec  7 13:41 ..
lrwxr-xr-x   1 root  admin    36B Dec  7 13:41 .VolumeIcon.icns -> System/Volumes/Data/.VolumeIcon.icns
----------   1 root  admin     0B Dec  7 13:41 .file
drwxr-xr-x   2 root  wheel    64B Dec  7 13:41 .nofollow
drwxr-xr-x   2 root  wheel    64B Dec  7 13:41 .resolve
drwxr-xr-x   2 root  wheel    64B Dec  7 13:41 .vol
drwxrwxr-x  26 root  admin   832B Jan 12 23:51 Applications
drwxr-xr-x  67 root  wheel   2.1K Dec 28 11:26 Library
drwxr-xr-x@ 10 root  wheel   320B Dec  7 13:41 System
drwxr-xr-x   5 root  admin   160B Dec 28 11:26 Users
drwxr-xr-x   4 root  wheel   128B Jan 10 09:43 Volumes
drwxr-xr-x@ 39 root  wheel   1.2K Dec  7 13:41 bin
drwxr-xr-x   2 root  wheel    64B Jul 14  2022 cores
dr-xr-xr-x   4 root  wheel   4.6K Dec 28 11:25 dev
lrwxr-xr-x@  1 root  wheel    11B Dec  7 13:41 etc -> private/etc
lrwxr-xr-x   1 root  wheel    25B Dec 28 11:27 home -> /System/Volumes/D

In [18]:
# Tokenize the input
tokenized_text = tokenizer(input_text)
pprint(tokenized_text)
# the tokenizer that i have borrowed from distilbert model, i am using that and converting my input to set of tokens
# there are 2 keys 1) attention mask 2) input_ids
# in input_ids we can see those 3 sentences that are converted to set of numbers, that is nlp
# all sentences are starting from 101 and ending at 102, these are special tokens

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1]],
 'input_ids': [[101, 7632, 2026, 2171, 2003, 14855, 2721, 3501, 102],
               [101, 1045, 2572, 4083, 17662, 2227, 1012, 102],
               [101, 1045, 2293, 17662, 2227, 102]]}


In [28]:
type(tokenized_text['input_ids'])
# this cannot go as a list to model it needs to be converted to tensor

list

In [15]:
print(tokenizer.convert_ids_to_tokens([101,102,7632]))

['[CLS]', '[SEP]', 'hi']


In [16]:
print(tokenizer.all_special_tokens)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']


In [17]:
print(tokenizer.all_special_tokens_extended)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']


In [23]:
print(tokenizer.convert_ids_to_tokens(tokenized_text['input_ids'][1]))

['[CLS]', 'i', 'am', 'learning', 'hugging', 'face', '.', '[SEP]']


In [32]:
tokenized_text = tokenizer(input_text,padding=True,return_tensors='pt')
pprint(tokenized_text)
# equal size for all sentences are needed so we use padding, corresponding att weights are 0

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0]]),
 'input_ids': tensor([[  101,  7632,  2026,  2171,  2003, 14855,  2721,  3501,   102],
        [  101,  1045,  2572,  4083, 17662,  2227,  1012,   102,     0],
        [  101,  1045,  2293, 17662,  2227,   102,     0,     0,     0]])}


In [33]:
tokenizer(['jdnhjsndhjnsdkjbfjnskdhsnldjbhbnskdjc'])

{'input_ids': [[101, 26219, 25311, 22578, 4859, 2232, 22895, 16150, 2243, 3501, 29292, 22895, 6711, 16425, 2015, 20554, 2094, 3501, 23706, 24700, 6711, 2094, 3501, 2278, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [34]:
# dynamic and static padding
# how to pass the input to the model
from transformers import AutoModel
model = AutoModel.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [35]:
pprint(model)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [38]:
model.state_dict()
# weights for every parameter in this model

OrderedDict([('embeddings.word_embeddings.weight',
              tensor([[-0.0166, -0.0666, -0.0163,  ..., -0.0200, -0.0514, -0.0264],
                      [-0.0132, -0.0673, -0.0161,  ..., -0.0227, -0.0554, -0.0260],
                      [-0.0176, -0.0709, -0.0144,  ..., -0.0246, -0.0596, -0.0232],
                      ...,
                      [-0.0231, -0.0588, -0.0105,  ..., -0.0195, -0.0262, -0.0212],
                      [-0.0490, -0.0561, -0.0047,  ..., -0.0107, -0.0180, -0.0219],
                      [-0.0065, -0.0915, -0.0025,  ..., -0.0151, -0.0504,  0.0460]])),
             ('embeddings.position_embeddings.weight',
              tensor([[ 1.8007e-02, -2.3798e-02, -3.5982e-02,  ...,  4.5726e-04,
                        5.1363e-05,  1.5002e-02],
                      [ 7.8592e-03,  4.8144e-03, -1.6093e-02,  ...,  2.9312e-02,
                        2.7634e-02, -8.5431e-03],
                      [-1.1663e-02, -3.1590e-03, -9.4000e-03,  ...,  1.4870e-02,
                 

In [39]:
pprint(model.config)

DistilBertConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "distilbert-base-uncased-finetuned-sst-2-english",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.47.1",
  "vocab_size": 30522
}



In [40]:
pprint(model.config.id2label[1])

'POSITIVE'


In [41]:
print(model.num_parameters())

66362880


In [43]:
# how to get output from the model
output = model(**tokenized_text)
print(output.last_hidden_state)
# there are 3 outputs each corresponds to 1 sentence.each sentence has 768 values.

tensor([[[ 0.4989,  0.2650,  0.7224,  ...,  0.3056,  0.8426, -0.4946],
         [ 1.0943,  0.6738,  0.4387,  ...,  0.3247,  1.0548, -0.2910],
         [ 0.5534,  0.7122,  0.0708,  ..., -0.1588,  0.5520, -0.0979],
         ...,
         [ 0.6388,  0.1362,  0.9540,  ..., -0.1345,  0.3535, -1.0845],
         [ 1.0395,  0.2805,  0.9409,  ...,  0.2725,  0.2241, -0.7777],
         [ 0.9516,  0.1733,  0.2277,  ...,  0.6889, -0.0193, -0.6055]],

        [[-0.2252,  0.2171,  0.4768,  ...,  0.1197,  0.8128, -0.2198],
         [ 0.1694,  0.5226,  0.3679,  ...,  0.1092,  0.8376, -0.1646],
         [-0.1130,  0.6109,  0.4617,  ...,  0.0256,  0.5946, -0.1023],
         ...,
         [-0.2478,  0.1569,  0.4603,  ...,  0.3068,  0.8271, -0.3650],
         [ 0.8436,  0.4379,  0.3740,  ...,  0.5070,  0.4120, -0.8064],
         [-0.3192,  0.2825,  0.3729,  ...,  0.3058,  0.8100, -0.0756]],

        [[ 0.3052,  0.3686,  0.2877,  ...,  0.4137,  0.9488, -0.4322],
         [ 0.5415,  0.4783,  0.0886,  ...,  0

In [44]:
print(output.last_hidden_state.shape)

torch.Size([3, 9, 768])


In [46]:
with torch.inference_mode():
    output = model(**tokenized_text)
    print(output)

BaseModelOutput(last_hidden_state=tensor([[[ 0.4989,  0.2650,  0.7224,  ...,  0.3056,  0.8426, -0.4946],
         [ 1.0943,  0.6738,  0.4387,  ...,  0.3247,  1.0548, -0.2910],
         [ 0.5534,  0.7122,  0.0708,  ..., -0.1588,  0.5520, -0.0979],
         ...,
         [ 0.6388,  0.1362,  0.9540,  ..., -0.1345,  0.3535, -1.0845],
         [ 1.0395,  0.2805,  0.9409,  ...,  0.2725,  0.2241, -0.7777],
         [ 0.9516,  0.1733,  0.2277,  ...,  0.6889, -0.0193, -0.6055]],

        [[-0.2252,  0.2171,  0.4768,  ...,  0.1197,  0.8128, -0.2198],
         [ 0.1694,  0.5226,  0.3679,  ...,  0.1092,  0.8376, -0.1646],
         [-0.1130,  0.6109,  0.4617,  ...,  0.0256,  0.5946, -0.1023],
         ...,
         [-0.2478,  0.1569,  0.4603,  ...,  0.3068,  0.8271, -0.3650],
         [ 0.8436,  0.4379,  0.3740,  ...,  0.5070,  0.4120, -0.8064],
         [-0.3192,  0.2825,  0.3729,  ...,  0.3058,  0.8100, -0.0756]],

        [[ 0.3052,  0.3686,  0.2877,  ...,  0.4137,  0.9488, -0.4322],
         [ 

In [47]:
output.last_hidden_state.shape

torch.Size([3, 9, 768])

In [48]:
# lets use it for classification
from transformers import AutoModelForSequenceClassification
classification_model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

In [49]:
print(classification_model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [50]:
# look at the last layer, 786 output is project onto 2 neurons for binary classification task
print(model)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [52]:
logits = classification_model(**tokenized_text)

In [53]:
probab_scores = torch.nn.functional.softmax(logits.logits,dim=1)
probab_scores

tensor([[5.9102e-03, 9.9409e-01],
        [9.2098e-04, 9.9908e-01],
        [1.5445e-04, 9.9985e-01]], grad_fn=<SoftmaxBackward0>)

In [55]:
pred_labels = torch.argmax(probab_scores,dim=1)
pred_labels
# all these 3 sentences are +ve sentiments

tensor([1, 1, 1])

In [56]:
#HF abstraction
from transformers import TextClassificationPipeline
aclassifier = TextClassificationPipeline(model=classification_model, tokenizer=tokenizer)

Device set to use mps:0


In [57]:
aclassifier(input_text)

[{'label': 'POSITIVE', 'score': 0.9940898418426514},
 {'label': 'POSITIVE', 'score': 0.9990789890289307},
 {'label': 'POSITIVE', 'score': 0.9998455047607422}]

In [59]:
# high lvl of abstraction
from transformers import pipeline
classifier=pipeline('sentiment-analysis')
classifier('I hate u bro.')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use mps:0


[{'label': 'NEGATIVE', 'score': 0.999402642250061}]

In [60]:
generator=pipeline('text-generation')
generator('I have to skip college beacuse')

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use mps:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "I have to skip college beacuse all those things are stupid.\n\n\nBut I won't be trying again. Thanks for your concern about the future of women.\n\n\n(H/T the man who is a jerk on a blog...)"}]

In [61]:
dir(generator)

['XL_PREFIX',
 '__abstractmethods__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_batch_size',
 '_create_repo',
 '_ensure_tensor_on_device',
 '_forward',
 '_forward_params',
 '_get_files_timestamps',
 '_load_feature_extractor',
 '_load_image_processor',
 '_load_processor',
 '_load_tokenizer',
 '_num_workers',
 '_parse_and_tokenize',
 '_postprocess_params',
 '_preprocess_params',
 '_sanitize_parameters',
 '_upload_modified_files',
 'binary_output',
 'call_count',
 'check_model_type',
 'default_input_names',
 'device',
 'device_placement',
 'ensure_tensor_on_device',
 'feature_extractor',
 'forward',
 'framework',
 'generati

In [62]:
generator = pipeline(model='distilgpt2')
generator('I wish i could play with you')

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use mps:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "I wish i could play with you again later in the day...but to find my own. We love you. I wish you a lot. I'll be very happy if you've made a decision."}]

In [63]:
unmask_model = pipeline('fill-mask')

No model was supplied, defaulted to distilbert/distilroberta-base and revision fb53ab8 (https://huggingface.co/distilbert/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use mps:0


In [64]:
unmask_model('My mother was so <mask> that I has to leave',top_k=10)

[{'score': 0.4967096745967865,
  'token': 4904,
  'token_str': ' upset',
  'sequence': 'My mother was so upset that I has to leave'},
 {'score': 0.11791927367448807,
  'token': 5800,
  'token_str': ' angry',
  'sequence': 'My mother was so angry that I has to leave'},
 {'score': 0.052780937403440475,
  'token': 30719,
  'token_str': ' distraught',
  'sequence': 'My mother was so distraught that I has to leave'},
 {'score': 0.03677225857973099,
  'token': 7758,
  'token_str': ' mad',
  'sequence': 'My mother was so mad that I has to leave'},
 {'score': 0.02994026057422161,
  'token': 15940,
  'token_str': ' furious',
  'sequence': 'My mother was so furious that I has to leave'},
 {'score': 0.0196221973747015,
  'token': 11521,
  'token_str': ' devastated',
  'sequence': 'My mother was so devastated that I has to leave'},
 {'score': 0.017505336552858353,
  'token': 8265,
  'token_str': ' scared',
  'sequence': 'My mother was so scared that I has to leave'},
 {'score': 0.01356926001608371